In [46]:
!pip install kaggle

In [47]:
!mkdir -p ~/.kaggle

In [48]:
!cp kaggle.json ~/.kaggle/

In [49]:
!chmod 600 ~/.kaggle/kaggle.json

# **Importing Twitter Sentiment Dataset**

In [50]:
!kaggle datasets download -d kazanova/sentiment140

sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [51]:
# extract the compress dataset
from zipfile import ZipFile
data = '/content/sentiment140.zip'

In [52]:
with ZipFile(data,'r') as zip:
  zip.extractall()
  print("The data is extracted")

The data is extracted


# **Importing the Dependencies**

In [53]:
import numpy as np

In [54]:
import pandas as pd

In [55]:
import re

In [56]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [57]:
from sklearn.model_selection import train_test_split

In [58]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [59]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [60]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# **Data Processing**

In [61]:
twi_dat = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')

In [62]:
twi_dat.shape

(1599999, 6)

In [63]:
twi_dat.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [64]:
# Giving the column names
column_names = ['target','id','date','flag','user','text']

In [65]:
twi_dat = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names = column_names,encoding='ISO-8859-1')

In [66]:
twi_dat.shape

(1600000, 6)

In [67]:
twi_dat.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [68]:
# Counting the number of missing values
twi_dat.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [69]:
# Checking the distribution of target column
twi_dat['target'].value_counts()

0    800000
4    800000
Name: target, dtype: int64

0 --> Negative Tweets

4 --> Positive Tweets

**Stemming**

In [70]:
port_stem = PorterStemmer()

In [71]:
def stemming(content):
  stem_con = re.sub('[^a-zA-z]',' ',content)
  stem_con = stem_con.lower()
  stem_con = stem_con.split()
  stem_con = [port_stem.stem(word) for word in stem_con if not word in stopwords.words('english')]
  stem_con = ' '.join(stem_con)

  return stem_con

In [36]:
twi_dat['stem_con'] = twi_dat['text'].apply(stemming)

In [37]:
twi_dat['stem_con']

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stem_con, Length: 1600000, dtype: object

In [38]:
X = twi_dat['stem_con'].values
Y = twi_dat['target'].values

In [39]:
print(X)
print(Y)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']
[0 0 0 ... 4 4 4]


In [40]:
# Training and Spliting the Data

In [41]:
X_train, X_test, Y_train,Y_test = train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

# converting the textual data to numerical

In [42]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

# Logistic Regression

In [ ]:
model = LogisticRegression(max_iter=1000)

In [75]:
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

In [78]:
X_train_pred = model.predict(X_train)
train_acc = accuracy_score(Y_train,X_train_pred)

In [80]:
print("Accuracy of the training data : ",train_acc*100)

Accuracy of the training data :  81.16718750000001


In [82]:
X_test_pred = model.predict(X_test)
test_acc = accuracy_score(Y_test,X_test_pred)

In [83]:
print("Accuracy of the test data : ",test_acc*100)

Accuracy of the test data :  77.8546875


In [84]:
import pickle

In [85]:
filename = 'trained.sav'
pickle.dump(model,open(filename,'wb'))